In [1]:
from src.textclf_transformer import *

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float16 if device == 'cuda' else torch.float32
print(f"Używane urządzenie: {device}, dtype: {dtype}")

Używane urządzenie: cpu, dtype: torch.float32


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

# Ustawienia Globalne
EMBED_DIM = 128
ATTN_EMBED_DIM = 128
NUM_HEADS = 8
BATCH_SIZE = 1
NUM_WARMUP = 10
NUM_RUNS = 10
HEAD_DIM = EMBED_DIM // NUM_HEADS




favor = AttentionBlock(
    embedding_dim=EMBED_DIM, num_heads=NUM_HEADS, projection_bias=True,
    attn_dropout=0.0, out_dropout=0.0, attention_kind='favor',
    attention_params={'nb_features': 64, 'redraw_interval': 0, 'ortho_features': True, 'phi': 'exp', 'stabilize': True, 'eps': 1.0e-6}
).to(device).to(dtype).eval()

lsh = AttentionBlock(
    embedding_dim=EMBED_DIM, num_heads=NUM_HEADS, projection_bias=True,
    attn_dropout=0.0, out_dropout=0.0, attention_kind='lsh',
    attention_params={'num_hashes': 2, 'chunk_size': 64, 'mask_within_chunks': True}
).to(device).to(dtype).eval()

flash = nn.MultiheadAttention(embed_dim=EMBED_DIM, num_heads=NUM_HEADS, dropout=0.0,
                            bias=True, batch_first=True, device=device).to(dtype).eval()

mha = AttentionBlock(
    embedding_dim=EMBED_DIM,  num_heads=NUM_HEADS, projection_bias=True,
    attn_dropout=0.0, out_dropout=0.0, attention_kind='mha',
    attention_params={}
).to(device).to(dtype).eval()


# ==============================================================================
# 2. FUNKCJE POMOCNICZE
# ==============================================================================

def rand_inputs(seq_len, device, dtype):
    x = torch.randn(BATCH_SIZE, seq_len, EMBED_DIM, device=device, dtype=dtype)
    mask = torch.rand(BATCH_SIZE, seq_len, device=device) < 0.1
    return x, mask


def time_run(func, x, key_padding_mask, num_runs, num_warmup):
    """Przeprowadza rozgrzewkę i mierzy czas uśredniony (ms)."""
    if device == 'cuda':
        torch.cuda.synchronize()
        torch.cuda.empty_cache()

    # WARMUP
    for _ in range(num_warmup):
        _ = func(x, key_padding_mask)
    if device == 'cuda':
        torch.cuda.synchronize()

    # POMIAR
    if device == 'cuda':
        start_event = torch.cuda.Event(enable_timing=True)
        end_event = torch.cuda.Event(enable_timing=True)
        start_event.record()
        for _ in range(num_runs):
            _ = func(x, key_padding_mask)
        end_event.record()
        torch.cuda.synchronize()
        total_time_ms = start_event.elapsed_time(end_event)
    else:
        start_time = time.time()
        for _ in range(num_runs):
            _ = func(x, key_padding_mask)
        total_time_ms = (time.time() - start_time) * 1000
        print(total_time_ms)

    

    return total_time_ms / num_runs



SEQ_LEN_LIST = [128, 256, 512, 1024, 2048, 4096, 8192]

results = {
    "FAVOR+ (Twoja Impl.)": [],
    "LSH Attn (Twoja Impl.)": [],
    "SDPA (Native - Flash)": [],
    "SDPA (Native - Non Flash)": [],
    "SDPA (Ours)": [],
}

implementations = {
    "FAVOR+ (Twoja Impl.)": lambda x, kpm: favor(x, key_padding_mask=kpm),
    "LSH Attn (Twoja Impl.)": lambda x, kpm: lsh(x, key_padding_mask=kpm),
    "SDPA (Native - Flash)": lambda x, kpm: flash(x, x, x, need_weights=False,
                            key_padding_mask=kpm, average_attn_weights=False),
    "SDPA (Native - Non Flash)": lambda x, kpm: flash(x, x, x, need_weights=True,
                            key_padding_mask=kpm, average_attn_weights=False),
    "SDPA (Ours)": lambda x, kpm: mha(x, key_padding_mask=kpm),
}


with torch.no_grad():
    for seq_len in SEQ_LEN_LIST:
        print(f"Testowanie N = {seq_len}...")

        x, kpm = rand_inputs(seq_len, device=device, dtype=dtype)

        for name, func in implementations.items():

            avg_time = time_run(func, x, kpm, NUM_RUNS, NUM_WARMUP)

            results[name].append(avg_time)
            print(f"  {name:<30}: {avg_time:.4f} ms")


        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("GPU cache has been emptied.")
        else:
            print("CUDA is not available. No GPU cache to empty.")

print("\n--- Zakończono pomiary ---")

plt.figure(figsize=(12, 8))

for name, times in results.items():
    valid_times = [times[i] for i, x in enumerate(times) if not np.isnan(x)]
    valid_seq_lens = [SEQ_LEN_LIST[i] for i, x in enumerate(times) if not np.isnan(x)]

    # Dodanie stylów dla lepszej czytelności
    if "FAVOR" in name:
        style = {'color': 'blue', 'linestyle': '-', 'marker': 'o'}
    elif "LSH" in name:
        style = {'color': 'green', 'linestyle': '--', 'marker': 's'}
    elif "SDPA" in name:
        style = {'color': 'red', 'linestyle': '-', 'marker': '^'}
    else:
        style = {'color': 'orange', 'linestyle': ':', 'marker': 'D'}

    plt.plot(valid_seq_lens, valid_times, **style, label=name)


plt.title(f'Porównanie Czasu Wykonania 4 Implementacji Uwagi (Warmup={NUM_WARMUP}, Runs={NUM_RUNS})', fontsize=16)
plt.xlabel('Długość Sekwencji (N)', fontsize=14)
plt.ylabel(f'Czas Wykonania (ms)', fontsize=14)
plt.grid(True, which="both", ls="--")
plt.legend(fontsize=12, loc='upper left')
plt.tight_layout()
plt.show()